In [1]:
# import required libraries for dataframe and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from lifelines import KaplanMeierFitter

# Ensuring all rows and columns are visible
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:,.2f}'.format)

In [2]:
# Reading the data on which analysis needs to be done

df = pd.read_csv('cleaned_data.csv',index_col = 0)
df.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR
0,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-05,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
1,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-12-16,214.79,198.94,Esker,1,STANDARD DOMESTIC,"511,868,043.00",CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
2,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-26,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
3,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-10-19,429.58,397.89,Esker,2,STANDARD DOMESTIC,"511,849,315.00",CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
4,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-19,623.16,565.77,Online,3,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016


In [3]:
# set Transaction Date to datetime 
df['TRX_DATE'] = pd.to_datetime(df['TRX_DATE'])

In [4]:
df.shape

(380401, 35)

In [5]:
# read variables dataset
variables = pd.read_csv('variables_with_churn.csv',index_col = 0 )
variables.head()

,CUSTOMER_SITE_ID,Recency,Frequency,Num_of_Trxns,Avg_Trxn_Amt,Avg_Margin,Avg_Quantity,Mode_of_Product_Family,Mode_of_Product_Model,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,STRATEGIC_ACCOUNTS,TERRITORY_REGION,Tenure,Cluster_Id,churn_prob_BGNBD,Churned_365,Churned_BGNBD
0,24,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,GRAPHICS,60085,END USER,Industrial,S,None,144782380,13.00,7.00,6.00,Call,81.00,1.97,5.00,0.00,0.00,No Contract,0,MW,77.00,3,0.05,0,0
1,90,50,18.86,98,233.90,172.39,11.37,CIJ,MAKE-UP,1,3,EDI,EDI,1.35,CHEMICALS,65802,END USER,Industrial,S,None,43937895,57.00,13.00,44.00,Call,53.00,3.03,6.00,6.00,"1,003.00",FSMA,0,MC,"1,879.00",3,0.02,0,0
2,111,415,72.85,21,220.75,188.43,1.05,CIJ,MAKE-UP,1,2,EDI,EDI,1.28,GRAPHICS,56073,END USER,Industrial,S,None,119130057,1.00,0.00,1.00,Call,22.00,3.03,2.00,0.00,0.00,No Contract,0,MW,"1,457.00",1,0.49,1,1
3,114,6,11.38,174,145.43,103.28,4.43,CIJ,MAKE-UP,3,4,EDI,EDI,1.25,PHARMA & MEDICAL,92614,END USER,Industrial,M,None,84160407,57.00,14.00,43.00,Call,70.00,1.00,15.00,11.00,521.64,FSMA,0,NW,"1,974.00",3,0.00,0,0
4,126,49,61.94,19,835.78,605.25,8.84,TIJ,INK,1,1,EDI,EDI,0.75,PHARMA & MEDICAL,8822,END USER,Industrial,S,Over Stocked / Timing,36781508,1.00,0.00,1.00,Call,13.00,1.00,2.00,0.00,0.00,No Contract,0,NE,"1,164.00",2,0.01,0,0


In [6]:
variables.groupby('Churned_365')['Tenure'].mean()

Churned_365
0   1,521.21
1     620.54
Name: Tenure, dtype: float64

In [7]:
variables.shape

(8367, 38)

In [8]:
df.shape

(380401, 35)

In [9]:
variables.head()

,CUSTOMER_SITE_ID,Recency,Frequency,Num_of_Trxns,Avg_Trxn_Amt,Avg_Margin,Avg_Quantity,Mode_of_Product_Family,Mode_of_Product_Model,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,STRATEGIC_ACCOUNTS,TERRITORY_REGION,Tenure,Cluster_Id,churn_prob_BGNBD,Churned_365,Churned_BGNBD
0,24,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,GRAPHICS,60085,END USER,Industrial,S,None,144782380,13.00,7.00,6.00,Call,81.00,1.97,5.00,0.00,0.00,No Contract,0,MW,77.00,3,0.05,0,0
1,90,50,18.86,98,233.90,172.39,11.37,CIJ,MAKE-UP,1,3,EDI,EDI,1.35,CHEMICALS,65802,END USER,Industrial,S,None,43937895,57.00,13.00,44.00,Call,53.00,3.03,6.00,6.00,"1,003.00",FSMA,0,MC,"1,879.00",3,0.02,0,0
2,111,415,72.85,21,220.75,188.43,1.05,CIJ,MAKE-UP,1,2,EDI,EDI,1.28,GRAPHICS,56073,END USER,Industrial,S,None,119130057,1.00,0.00,1.00,Call,22.00,3.03,2.00,0.00,0.00,No Contract,0,MW,"1,457.00",1,0.49,1,1
3,114,6,11.38,174,145.43,103.28,4.43,CIJ,MAKE-UP,3,4,EDI,EDI,1.25,PHARMA & MEDICAL,92614,END USER,Industrial,M,None,84160407,57.00,14.00,43.00,Call,70.00,1.00,15.00,11.00,521.64,FSMA,0,NW,"1,974.00",3,0.00,0,0
4,126,49,61.94,19,835.78,605.25,8.84,TIJ,INK,1,1,EDI,EDI,0.75,PHARMA & MEDICAL,8822,END USER,Industrial,S,Over Stocked / Timing,36781508,1.00,0.00,1.00,Call,13.00,1.00,2.00,0.00,0.00,No Contract,0,NE,"1,164.00",2,0.01,0,0


In [10]:
# merge into a big dataset
df_final = pd.merge(df, variables, on='CUSTOMER_SITE_ID')
df_final.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL_x,POSTAL_CODE_x,CUSTOMER_CLASS_x,TERRITORY_TYPE_x,SUPPLIES_SEGMENTATION_x,SUPPLIES_DECLINE_REASONS_x,DUNS_NUMBER_x,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents_x,Total_Repeat_Calls_x,Total_FTF_Calls_x,Most_Frequent_Interaction_Type_x,Total_Visits_x,Total_Cases_x,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases_x,Total_Contracts_x,Contract_length_x,Contract_Category_x,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS_x,TERRITORY_REGION_x,TRX_YEAR,Recency,Frequency,Num_of_Trxns,Avg_Trxn_Amt,Avg_Margin,Avg_Quantity,Mode_of_Product_Family,Mode_of_Product_Model,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,SHORT_VERTICAL_y,POSTAL_CODE_y,CUSTOMER_CLASS_y,TERRITORY_TYPE_y,SUPPLIES_SEGMENTATION_y,SUPPLIES_DECLINE_REASONS_y,DUNS_NUMBER_y,Total_SVC_Incidents_y,Total_Repeat_Calls_y,Total_FTF_Calls_y,Most_Frequent_Interaction_Type_y,Total_Visits_y,Total_Cases_y,Num_of_Active_Install_Bases_y,Total_Contracts_y,Contract_length_y,Contract_Category_y,STRATEGIC_ACCOUNTS_y,TERRITORY_REGION_y,Tenure,Cluster_Id,churn_prob_BGNBD,Churned_365,Churned_BGNBD
0,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-05,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
1,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-26,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
2,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-19,623.16,565.77,Online,3,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
3,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-10,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
4,0.63,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-10-03,"1,102.72",993.27,Online,8,STANDARD DOMESTIC,"511,843,348.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1


In [11]:
df_final.columns = df_final.columns.str.replace('_x','',regex = True)

In [12]:
df_final.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR,Recency,Frequency,Num_of_Trxns,Avg_Trxn_Amt,Avg_Margin,Avg_Quantity,Mode_of_Product_Family,Mode_of_Product_Model,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,SHORT_VERTICAL_y,POSTAL_CODE_y,CUSTOMER_CLASS_y,TERRITORY_TYPE_y,SUPPLIES_SEGMENTATION_y,SUPPLIES_DECLINE_REASONS_y,DUNS_NUMBER_y,Total_SVC_Incidents_y,Total_Repeat_Calls_y,Total_FTF_Calls_y,Most_Frequent_Interaction_Type_y,Total_Visits_y,Total_Cases_y,Num_of_Active_Install_Bases_y,Total_Contracts_y,Contract_length_y,Contract_Category_y,STRATEGIC_ACCOUNTS_y,TERRITORY_REGION_y,Tenure,Cluster_Id,churn_prob_BGNBD,Churned_365,Churned_BGNBD
0,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-05,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
1,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-26,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
2,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-19,623.16,565.77,Online,3,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
3,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-10,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1
4,0.63,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-10-03,"1,102.72",993.27,Online,8,STANDARD DOMESTIC,"511,843,348.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,11.00,3.00,8.00,Call,18.00,1.97,6.00,0.00,0.00,No Contract,0,NW,"1,483.00",3,1.00,1,1


In [13]:
df_final.drop(['SHORT_VERTICAL_y',
       'POSTAL_CODE_y', 'CUSTOMER_CLASS_y', 'TERRITORY_TYPE_y',
       'SUPPLIES_SEGMENTATION_y', 'SUPPLIES_DECLINE_REASONS_y',
       'DUNS_NUMBER_y', 'Total_SVC_Incidents_y', 'Total_Repeat_Calls_y',
       'Total_FTF_Calls_y', 'Most_Frequent_Interaction_Type_y',
       'Total_Visits_y', 'Total_Cases_y',
       'Num_of_Active_Install_Bases_y', 'Total_Contracts_y',
       'Contract_length_y', 'Contract_Category_y',
       'STRATEGIC_ACCOUNTS_y', 'TERRITORY_REGION_y'], axis = 1,inplace = True)

In [14]:
df_final.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR,Recency,Frequency,Num_of_Trxns,Avg_Trxn_Amt,Avg_Margin,Avg_Quantity,Mode_of_Product_Family,Mode_of_Product_Model,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,Tenure,Cluster_Id,churn_prob_BGNBD,Churned_365,Churned_BGNBD
0,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-05,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,"1,483.00",3,1.00,1,1
1,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-26,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,"1,483.00",3,1.00,1,1
2,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-19,623.16,565.77,Online,3,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,"1,483.00",3,1.00,1,1
3,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-10,207.72,188.59,Online,1,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,"1,483.00",3,1.00,1,1
4,0.63,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-10-03,"1,102.72",993.27,Online,8,STANDARD DOMESTIC,"511,843,348.00",CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016,477,32.24,47,455.66,389.53,4.57,CIJ,MAKE-UP,1,3,Online,STANDARD DOMESTIC,0.80,"1,483.00",3,1.00,1,1


In [15]:
df_final.sort_values('CUSTOMER_SITE_ID',ascending = True, inplace=True)

In [16]:
df_final.reset_index(inplace=True)

In [17]:
df_final = df_final.iloc[:,1:]

In [18]:
df_final.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR,Recency,Frequency,Num_of_Trxns,Avg_Trxn_Amt,Avg_Margin,Avg_Quantity,Mode_of_Product_Family,Mode_of_Product_Model,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,Tenure,Cluster_Id,churn_prob_BGNBD,Churned_365,Churned_BGNBD
0,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-03-20,"1,855.74","1,381.20",Esker,6,STANDARD DOMESTIC,"512,254,720.00",LASER,FUME EXTRACTION,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
1,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-04-15,552.68,537.02,SFDC_CPQ,2,EQUIPMENT DOMESTIC,"512,260,527.00",LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
2,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-05-04,"5,000.00","4,843.37",Online,20,STANDARD DOMESTIC,"512,267,173.00",LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
3,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-05-11,"5,000.00","4,843.37",Copy,20,STANDARD DOMESTIC,"512,268,803.00",LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
4,1.40,37,90,CHEMICALS,65802,END USER,Industrial,S,None,43937895,2017-01-03,213.80,129.59,EDI,20,EDI,"511,872,093.00",CIJ,MAKE-UP,57.00,13.00,44.00,Call,53.00,3.03,unknown,unknown,6.00,6.00,"1,003.00",FSMA,0.00,0,MC,2017,50,18.86,98,233.90,172.39,11.37,CIJ,MAKE-UP,1,3,EDI,EDI,1.35,"1,879.00",3,0.02,0,0


In [19]:
df_final.shape

(380401, 53)

In [20]:
df_final.to_csv('analysis.csv')